<a href="https://colab.research.google.com/github/chestonriley/python_misc/blob/master/Consolidate_crew_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gspread gspread-dataframe

In [2]:
import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import datetime
import gspread_dataframe as gd

creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Functions

In [4]:
def to_int_funct(str_value):

  return_val = int(str(str_value).split(".")[0].replace(",",""))
  return return_val

In [5]:
def total_seconds_conv(time_value):

  time_list = time_value.split(":")
  hour_seconds = int(time_list[0]) * 3600
  min_seconds  = int(time_list[1]) * 60
  seconds      = int(time_list[2])

  return hour_seconds + min_seconds + seconds

In [6]:
def convert_to_date(date_string):
    try:
        # Parse the input string using the format "%m/%d/%Y"
        date_object = datetime.datetime.strptime(date_string, "%m/%d/%Y")
        return date_object
    except ValueError as e:
        print(f"Error: {e}")
        return None

In [7]:
def create_date_range_dataframe(start_date, end_date, names):
    # Generate date range between start_date and end_date
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    # Create a DataFrame with Cartesian product of names and dates
    cartesian_product = pd.MultiIndex.from_product([names, date_range], names=['rower', 'date'])
    result_df = pd.DataFrame(index=cartesian_product).reset_index()

    return result_df

# Read sheets

## Data Collection

In [8]:
g_sheets = gc.open('St X Crew Data Collection').sheet1
df = pd.DataFrame(g_sheets.get_all_records(head=1))

indexAge = df[ (df['rower'] == '') ].index
df.drop(indexAge , inplace=True)

In [9]:
df['total_meters'] = df['total_meters'].apply(to_int_funct)

In [10]:
df['total_seconds'] = df['total_time'].apply(total_seconds_conv)

In [11]:
df.drop(columns=['total_time'],inplace=True)

In [12]:
df['date'] = df['date'].apply(convert_to_date)

In [13]:
df = df.groupby(['rower','date','workout_type']).sum().reset_index()

In [14]:
df_data_collection = df.copy()

## Roster

In [15]:
g_sheets = gc.open('St X Crew Data Collection').worksheet('Roster')
df = pd.DataFrame(g_sheets.get_all_records(head=1))

In [16]:
df.drop(columns=['Rower Full Name'],inplace=True)

In [17]:
df.rename(columns={'Rower Nick Name':'rower'},inplace=True)

In [18]:
df_roster = df.copy()

## Weights

In [19]:
g_sheets = gc.open('St X Crew Data Collection').worksheet('weigh_in')
df = pd.DataFrame(g_sheets.get_all_records(head=1))

In [20]:
df.rename(columns={'Rower Nick Name':'rower'
                  ,'Weight':'weight'
                  ,'Weigh_in_date':'weigh_in_date'}
          ,inplace=True)

In [21]:
df['weigh_in_date'] = df['weigh_in_date'].apply(convert_to_date)

In [22]:
df_weights = df.copy()

# Merge Data

In [23]:
min_date = df_data_collection['date'].min()
max_date = df_data_collection['date'].max()
print(min_date, max_date)

2023-10-01 00:00:00 2023-12-22 00:00:00


In [24]:
rower_name_list = df_roster['rower'].to_list()

## Set up the dataframe to do daily tracking

In [25]:
tracking_df = create_date_range_dataframe(min_date,max_date,rower_name_list)
tracking_df

,rower,date
0,William J,2023-10-01
1,William J,2023-10-02
2,William J,2023-10-03
3,William J,2023-10-04
4,William J,2023-10-05
...,...,...
1572,Colin H,2023-12-18
1573,Colin H,2023-12-19
1574,Colin H,2023-12-20
1575,Colin H,2023-12-21


## Merge in data collection

In [26]:
step_1_df = tracking_df.merge(df_data_collection,how='left',left_on=['rower','date'],right_on=['rower','date'])
step_1_df

,rower,date,workout_type,total_meters,total_seconds
0,William J,2023-10-01,NaN,NaN,NaN
1,William J,2023-10-02,NaN,NaN,NaN
2,William J,2023-10-03,NaN,NaN,NaN
3,William J,2023-10-04,NaN,NaN,NaN
4,William J,2023-10-05,NaN,NaN,NaN
...,...,...,...,...,...
1573,Colin H,2023-12-18,Test,2000.0,490.0
1574,Colin H,2023-12-19,NaN,NaN,NaN
1575,Colin H,2023-12-20,NaN,NaN,NaN
1576,Colin H,2023-12-21,NaN,NaN,NaN


## Merge in squad data

In [27]:
step_2_df = step_1_df.merge(df_roster,how='left',right_on='rower',left_on='rower')
step_2_df

,rower,date,workout_type,total_meters,total_seconds,Squad
0,William J,2023-10-01,NaN,NaN,NaN,Mavericks
1,William J,2023-10-02,NaN,NaN,NaN,Mavericks
2,William J,2023-10-03,NaN,NaN,NaN,Mavericks
3,William J,2023-10-04,NaN,NaN,NaN,Mavericks
4,William J,2023-10-05,NaN,NaN,NaN,Mavericks
...,...,...,...,...,...,...
1573,Colin H,2023-12-18,Test,2000.0,490.0,Sea Dragons
1574,Colin H,2023-12-19,NaN,NaN,NaN,Sea Dragons
1575,Colin H,2023-12-20,NaN,NaN,NaN,Sea Dragons
1576,Colin H,2023-12-21,NaN,NaN,NaN,Sea Dragons


## Create the weight df to use to join back

In [28]:
weight_tracking_df = create_date_range_dataframe(min_date,max_date,rower_name_list)
weight_tracking_df = weight_tracking_df.merge(df_weights,how='left',left_on='rower',right_on='rower')
weight_tracking_df['date_diff'] = (weight_tracking_df['weigh_in_date'] - weight_tracking_df['date']).dt.days
weight_tracking_df

,rower,date,weight,weigh_in_date,date_diff
0,William J,2023-10-01,120,2023-10-26,25
1,William J,2023-10-01,125,2023-11-09,39
2,William J,2023-10-02,120,2023-10-26,24
3,William J,2023-10-02,125,2023-11-09,38
4,William J,2023-10-03,120,2023-10-26,23
...,...,...,...,...,...
2153,Colin H,2023-12-18,170,2023-11-09,-39
2154,Colin H,2023-12-19,170,2023-11-09,-40
2155,Colin H,2023-12-20,170,2023-11-09,-41
2156,Colin H,2023-12-21,170,2023-11-09,-42


### Get rid of records that are when the date was before the weigh in

In [29]:
weight_index = weight_tracking_df[ (weight_tracking_df['date_diff'] > 0) ].index
weight_tracking_df.drop(weight_index , inplace=True)

### Get the indexes of the max records that we want to keep
And create the final adjusted weight dataframe

In [30]:
idx = weight_tracking_df.groupby(['rower', 'date'])['date_diff'].transform(max) == weight_tracking_df['date_diff']

In [31]:
final_weight_df = weight_tracking_df[idx].copy()
final_weight_df.drop(columns=['date_diff'],inplace=True)

## Merge in weight data

In [32]:
step_3_df = step_2_df.merge(final_weight_df,how='left',right_on=['rower','date'],left_on=['rower','date'])
step_3_df

,rower,date,workout_type,total_meters,total_seconds,Squad,weight,weigh_in_date
0,William J,2023-10-01,NaN,NaN,NaN,Mavericks,NaN,NaT
1,William J,2023-10-02,NaN,NaN,NaN,Mavericks,NaN,NaT
2,William J,2023-10-03,NaN,NaN,NaN,Mavericks,NaN,NaT
3,William J,2023-10-04,NaN,NaN,NaN,Mavericks,NaN,NaT
4,William J,2023-10-05,NaN,NaN,NaN,Mavericks,NaN,NaT
...,...,...,...,...,...,...,...,...
1573,Colin H,2023-12-18,Test,2000.0,490.0,Sea Dragons,170.0,2023-11-09
1574,Colin H,2023-12-19,NaN,NaN,NaN,Sea Dragons,170.0,2023-11-09
1575,Colin H,2023-12-20,NaN,NaN,NaN,Sea Dragons,170.0,2023-11-09
1576,Colin H,2023-12-21,NaN,NaN,NaN,Sea Dragons,170.0,2023-11-09


In [33]:
step_3_df.to_csv("out_rower_test.csv")

# Put in missing values

In [34]:
stage_1_df = step_3_df.copy()

In [35]:
stage_1_df['workout_type'] = stage_1_df['workout_type'].apply(lambda x:"None" if pd.isnull(x) else x)

In [36]:
stage_1_df['total_meters'] = stage_1_df['total_meters'].apply(lambda x:0 if pd.isnull(x) else x)

In [37]:
stage_1_df['total_seconds'] = stage_1_df['total_seconds'].apply(lambda x:0 if pd.isnull(x) else x)

## Put in the weight adjustment factor

In [38]:
stage_1_df['weight_adjustment_factor'] = (stage_1_df['weight']/270)**.222

## Apply rolling 14 day totals

In [39]:
stage_1_df['rolling_14_day_meters'] = stage_1_df.groupby('rower')['total_meters'].rolling(window=14,min_periods=1).sum().reset_index(level=0,drop=True)

In [40]:
stage_1_df['rolling_14_day_seconds'] = stage_1_df.groupby('rower')['total_seconds'].rolling(window=14,min_periods=1).sum().reset_index(level=0,drop=True)

In [41]:
stage_1_df['rolling_14_day_raw_500m_split_seconds'] = (stage_1_df['rolling_14_day_seconds']*500) / stage_1_df['rolling_14_day_meters']

In [42]:
stage_1_df['cumulative_total_meters'] = stage_1_df.groupby('rower')['total_meters'].rolling(window=10000,min_periods=1).sum().reset_index(level=0,drop=True)

In [43]:
stage_1_df['cumulative_total_seconds'] = stage_1_df.groupby('rower')['total_seconds'].rolling(window=10000,min_periods=1).sum().reset_index(level=0,drop=True)

In [44]:
stage_1_df['cumulative_raw_500m_split_seconds'] = (stage_1_df['cumulative_total_seconds']*500) / stage_1_df['cumulative_total_meters']

In [45]:
stage_1_df['wf_rolling_14_day_meters'] = stage_1_df['rolling_14_day_meters']/stage_1_df['weight_adjustment_factor']

In [46]:
stage_1_df['wf_rolling_14_day_seconds'] = stage_1_df['rolling_14_day_seconds']*stage_1_df['weight_adjustment_factor']

In [47]:
stage_1_df['wf_rolling_14_day_raw_500m_split_seconds'] = (stage_1_df['wf_rolling_14_day_seconds']*500) / stage_1_df['wf_rolling_14_day_meters']

In [48]:
stage_1_df['wf_cumulative_total_meters'] = stage_1_df['cumulative_total_meters']/stage_1_df['weight_adjustment_factor']

In [49]:
stage_1_df['wf_cumulative_total_seconds'] = stage_1_df['cumulative_total_seconds']*stage_1_df['weight_adjustment_factor']

In [50]:
stage_1_df['wf_cumulative_raw_500m_split_seconds'] = (stage_1_df['wf_cumulative_total_seconds']*500) / stage_1_df['wf_cumulative_total_meters']

# Apply ranks

In [51]:
ranking_dict = {
    'rolling_14_day_meters':'desc'
    ,'rolling_14_day_raw_500m_split_seconds':'asc'
    ,'cumulative_total_meters':'desc'
    ,'cumulative_raw_500m_split_seconds':'asc'
    ,'wf_rolling_14_day_meters':'desc'
    ,'wf_rolling_14_day_raw_500m_split_seconds':'asc'
    ,'wf_cumulative_total_meters':'desc'
    ,'wf_cumulative_raw_500m_split_seconds':'asc'
}

In [52]:
for ranking_field in ranking_dict:

  rank_order = ranking_dict[ranking_field]

  if rank_order == 'desc':
    stage_1_df[f"rank_{ranking_field}"] = stage_1_df.groupby(['date'])[ranking_field].rank(method='max',ascending=False)
  else:
    stage_1_df[f"rank_{ranking_field}"] = stage_1_df.groupby(['date'])[ranking_field].rank(method='max',ascending=True)


In [53]:
stage_1_df

,rower,date,workout_type,total_meters,total_seconds,Squad,weight,weigh_in_date,weight_adjustment_factor,rolling_14_day_meters,...,wf_cumulative_total_seconds,wf_cumulative_raw_500m_split_seconds,rank_rolling_14_day_meters,rank_rolling_14_day_raw_500m_split_seconds,rank_cumulative_total_meters,rank_cumulative_raw_500m_split_seconds,rank_wf_rolling_14_day_meters,rank_wf_rolling_14_day_raw_500m_split_seconds,rank_wf_cumulative_total_meters,rank_wf_cumulative_raw_500m_split_seconds
0,William J,2023-10-01,None,0.0,0.0,Mavericks,NaN,NaT,NaN,0.0,...,NaN,NaN,19.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN
1,William J,2023-10-02,None,0.0,0.0,Mavericks,NaN,NaT,NaN,0.0,...,NaN,NaN,19.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN
2,William J,2023-10-03,None,0.0,0.0,Mavericks,NaN,NaT,NaN,0.0,...,NaN,NaN,19.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN
3,William J,2023-10-04,None,0.0,0.0,Mavericks,NaN,NaT,NaN,0.0,...,NaN,NaN,19.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN
4,William J,2023-10-05,None,0.0,0.0,Mavericks,NaN,NaT,NaN,0.0,...,NaN,NaN,19.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,Colin H,2023-12-18,Test,2000.0,490.0,Sea Dragons,170.0,2023-11-09,0.902395,26190.0,...,7427.617086,115.998892,5.0,11.0,16.0,11.0,5.0,13.0,16.0,11.0
1574,Colin H,2023-12-19,None,0.0,0.0,Sea Dragons,170.0,2023-11-09,0.902395,26190.0,...,7427.617086,115.998892,3.0,12.0,16.0,11.0,3.0,14.0,16.0,11.0
1575,Colin H,2023-12-20,None,0.0,0.0,Sea Dragons,170.0,2023-11-09,0.902395,19315.0,...,7427.617086,115.998892,8.0,12.0,16.0,11.0,7.0,14.0,16.0,11.0
1576,Colin H,2023-12-21,None,0.0,0.0,Sea Dragons,170.0,2023-11-09,0.902395,14482.0,...,7427.617086,115.998892,9.0,9.0,16.0,11.0,9.0,13.0,16.0,11.0


In [54]:
stage_1_df['daily_avg_rank_rolling_14_day'] = (stage_1_df['rank_rolling_14_day_meters'] + stage_1_df['rank_wf_rolling_14_day_raw_500m_split_seconds']) / 2
stage_1_df[f"rank_final_rolling_14_day"] = stage_1_df.groupby(['date'])['daily_avg_rank_rolling_14_day'].rank(method='max',ascending=True)

In [55]:
max_rank = stage_1_df['rank_final_rolling_14_day'].max()

rank_adjust_list = list(ranking_dict.keys())
rank_adjust_list.append('daily_avg_rank_rolling_14_day')
rank_adjust_list.append('rank_final_rolling_14_day')

for column in rank_adjust_list:
  stage_1_df[column] = stage_1_df[column].fillna(value=max_rank)



# Output the data

In [56]:
stage_1_df.to_csv("/content/drive/MyDrive/rower_data_refined.csv",index=False)

In [57]:
sheet_name_out = 'rower_refined_data_workbook'
sh = gc.open(sheet_name_out)
worksheet = sh.get_worksheet(0)
worksheet.clear()

{'spreadsheetId': '1jWkTusWpaWTrBZPnkz7USDgtR81l-qqaZf28tkcZTEw',
 'clearedRange': 'Sheet1!A1:AE1579'}

In [58]:
gd.set_with_dataframe(worksheet,stage_1_df)